In [8]:
data_source = ['math-amc', 'math-aime', 'gpqa', 'mip-insufficient', 'mip-question', 'bbh', 'bbh_boolean_expressions', 'bbh_causal_judgement', 'bbh_date_understanding', 'bbh_disambiguation_qa', 'bbh_logical_deduction_five_objects', 'bbh_movie_recommendation', 'bbh_navigate', 'bbh_object_counting', 'bbh_snarks', 'underthink-bench', ]

In [2]:
import os

os.environ['HF_HOME'] = '/nas-ssd2/joykirat/.cache/huggingface'
os.environ['CUDA_VISIBLE_DEVICES'] = '7'
from transformers import AutoTokenizer, AutoModelForCausalLM
from vllm import LLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-4B")
# Qwen/Qwen3-4B
# deepseek-ai/DeepSeek-R1-Distill-Llama-8B
# deepseek-ai/DeepSeek-R1-Distill-Qwen-7B
# model = LLM(model="Qwen/Qwen3-4B", gpu_memory_utilization=0.95, tensor_parallel_size=1, task="embed")





INFO 10-01 09:38:55 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 10-01 09:38:55 [__init__.py:239] Automatically detected platform cuda.


In [1]:
import json
data = []
with open('/nas-ssd2/joykirat/code/verl-fork/verl/TTS_attention/rollouts.jsonl', 'r') as f:
    for line in f:
        data.append(json.loads(line))
        

In [3]:
import numpy as np

accuracy_by_source = {}
for line in data:
    if line['data_source'] not in accuracy_by_source:
        if 'bbh_' in line['data_source']:
            line['data_source'] = 'bbh'
        accuracy_by_source[line['data_source']] = []
    accuracy_by_source[line['data_source']].append(line['final_reward'])
    
for source in accuracy_by_source:
    mean_accuracy = np.mean(accuracy_by_source[source])
    print(source, mean_accuracy, len(accuracy_by_source[source]))
    
    

math-amc 0.636144578313253 415
gpqa 0.3715736040609137 985
math-aime 0.24269662921348314 445


In [4]:
# import numpy as np
# response_length_list = {}
# from tqdm import tqdm
# for line in tqdm(data):
#     if line['data_source'] not in response_length_list:
#         response_length_list[line['data_source']] = []
#     response_length_list[line['data_source']].append(len(tokenizer.encode(line['response'])))
# for source in response_length_list:
#     mean_response_length = np.mean(response_length_list[source])
#     print(source, mean_response_length)
            

In [29]:
import re
from math_verify.errors import TimeoutException
from math_verify.metric import math_metric
from math_verify.parser import ExprExtractionConfig, LatexExtractionConfig

def clean_prediction(model_output: str) -> str:
    """
    Try to normalize model outputs like 'Answer: C. Cotton' into '\boxed{C}'.
    """

    # If boxed answer is already there, keep it
    boxed_match = re.search(r'\\boxed\{([A-Za-z0-9]+)\}', model_output)
    if boxed_match:
        return f"\\boxed{{{boxed_match.group(1)}}}"

    # Otherwise, try to find a single-letter multiple-choice answer
    choice_match = re.search(r'\b([A-E])\b', model_output)
    if choice_match:
        return f"\\boxed{{{choice_match.group(1)}}}"

    # If nothing found, just return as-is
    return model_output.strip()

def getRawCorrectness(model_output: str, ground_truth: str) -> float:
    verify_func = math_metric(
        gold_extraction_target=(LatexExtractionConfig(),),
        pred_extraction_target=(ExprExtractionConfig(), LatexExtractionConfig()),
    )

    ret_score = 0.0
    ground_truth_boxed = f"\\boxed{{{ground_truth}}}"

    try:
        pred_clean = clean_prediction(model_output)
        ret_score, _ = verify_func([ground_truth_boxed], [pred_clean])
    except Exception:
        pass

    return ret_score

def getCorrectness(model_output: str, ground_truth: str) -> float:
    if '</think>' in model_output:
        model_output = model_output.split('</think>')[-1]

    verify_func = math_metric(
        gold_extraction_target=(LatexExtractionConfig(),),
        pred_extraction_target=(ExprExtractionConfig(), LatexExtractionConfig()),
    )

    ret_score = 0.0
    ground_truth_boxed = f"\\boxed{{{ground_truth}}}"

    try:
        pred_clean = clean_prediction(model_output)
        ret_score, _ = verify_func([ground_truth_boxed], [pred_clean])
    except Exception:
        pass

    return ret_score


In [30]:
line

{'data_source': 'math-aime',
 'prompt': "user\nLet $\\triangle ABC$ be an isosceles triangle with $\\angle A = 90^\\circ.$ There exists a point $P$ inside $\\triangle ABC$ such that $\\angle PAB = \\angle PBC = \\angle PCA$ and $AP = 10.$ Find the area of $\\triangle ABC.$ Let's think step by step and output the final answer within \\boxed{}.\nassistant\n",
 'response': "<think>\nOkay, so I need to find the area of triangle ABC, which is an isosceles right triangle with a right angle at A. There's a point P inside the triangle such that the angles PAB, PBC, and PCA are all equal. Also, AP is given as 10 units. Hmm, okay, let me try to visualize this.\n\nFirst, let me sketch the triangle. Since it's an isosceles right triangle with the right angle at A, sides AB and AC must be equal in length. Let me denote AB = AC = x, and BC would be the hypotenuse, so its length is x√2. The area of the triangle would then be (AB * AC)/2 = (x^2)/2. So if I can find x, I can compute the area.\n\nNow, t

In [31]:
import numpy as np
correctness_response_length_list = {}
from tqdm import tqdm
for line in tqdm(data):
    if line['data_source'] not in correctness_response_length_list:
        correctness_response_length_list[line['data_source']] = []
    question = line['prompt']
    correctness_response_length_list[line['data_source']].append((question, getCorrectness(line['response'], line['ground_truth']), len(tokenizer.encode(line['response']))))
    
            
            
            

100%|██████████| 1845/1845 [00:20<00:00, 88.58it/s] 


In [32]:
def compute_aucoaa(data):
    # data is a list of (correctness, response_length) tuples
    n = len(data)
    tmax = max(length for _,_, length in data)
    oaa_t_list = []
    for t in range(tmax + 1):
        filtered = [correct for _,correct, length in data if length < t]
        if filtered:
            oaa_t = sum(filtered) / n  # Denominator is always n, per definition
        else:
            oaa_t = 0.0
        oaa_t_list.append(oaa_t)
    aucoaa = sum(oaa_t_list) / len(oaa_t_list)
    return aucoaa


In [33]:
import numpy as np
from collections import defaultdict

def compute_accuracy_from_runs(prompts, accuracies, n_runs=5):
    """
    prompts: list of prompt IDs (each repeated n_runs times)
    accuracies: list of 0/1, same length as prompts
    n_runs: number of answers per prompt (default 5)
    
    Returns:
        dict with per-answer, per-prompt, and per-run accuracy + std
    """
    assert len(prompts) == len(accuracies), "Both lists must have the same length"

    accuracies = np.array(accuracies)
    total_answers = len(accuracies)

    # ------------------
    # Per-answer accuracy (binomial estimate)
    # ------------------
    acc_answer = accuracies.mean()
    std_answer = np.sqrt(acc_answer * (1 - acc_answer) / total_answers)

    # ------------------
    # Per-prompt accuracy
    # ------------------
    prompt_correct = defaultdict(list)
    for p, a in zip(prompts, accuracies):
        prompt_correct[p].append(a)

    prompt_solved = np.array([np.mean(v) for v in prompt_correct.values()])
    acc_prompt = prompt_solved.mean()
    std_prompt = prompt_solved.std(ddof=1)

    # ------------------
    # Per-run accuracy (like papers report)
    # ------------------
    # initialize n_runs lists (one per run)
    run_correct = [[] for _ in range(n_runs)]

    for p in set(prompts):
        answers = prompt_correct[p]   # length = n_runs
        for i, a in enumerate(answers):
            run_correct[i].append(a)

    run_accs = np.array([np.mean(r) for r in run_correct])
    acc_run = run_accs.mean()
    std_run = run_accs.std(ddof=1)

    return {
        "per_run_accuracy": acc_run,
        "per_run_std": std_run,
    }


In [34]:
for key in correctness_response_length_list.keys():
    print("Data Source: ", key)
    prompt_list = [prompt for prompt, _, _ in correctness_response_length_list[key]]
    accuracy_list = [correct for _, correct, _ in correctness_response_length_list[key]]
    response_length_list = [length for _, _, length in correctness_response_length_list[key]]

    result = compute_accuracy_from_runs(prompt_list, accuracy_list)
    print(f"Accuracy: {result['per_run_accuracy']}")
    print(f"Std: {result['per_run_std']}")
    print(f"Response Length: {np.mean(response_length_list)}")
    print(f"Max Response Length: {max(response_length_list)}")
    print(f"Min Response Length: {min(response_length_list)}")

    print("AUC-OAA: ", compute_aucoaa(correctness_response_length_list[key]))
    print("--------------------------------")
    
    # print(result)
    

Data Source:  math-amc
Accuracy: 0.6819277108433734
Std: 0.01827198902181095
Response Length: 7030.110843373494
Max Response Length: 10000
Min Response Length: 1309
AUC-OAA:  0.2948326854061582
--------------------------------
Data Source:  gpqa
Accuracy: 0.4517766497461929
Std: 0.023809217054941257
Response Length: 7625.045685279188
Max Response Length: 10002
Min Response Length: 1193
AUC-OAA:  0.16018077825383348
--------------------------------
Data Source:  math-aime
Accuracy: 0.27640449438202247
Std: 0.0170401695371945
Response Length: 9192.880898876405
Max Response Length: 10001
Min Response Length: 2631
AUC-OAA:  0.07978673928135721
--------------------------------


In [ ]:
0.7951807228915663
{'per_answer_accuracy': np.float64(0.7951807228915663), 'per_answer_std': np.float64(0.019810447580330987), 'per_prompt_accuracy': np.float64(0.7951807228915663), 'per_prompt_std': np.float64(0.35334761936606585)}
0.4720812182741117
{'per_answer_accuracy': np.float64(0.4720812182741117), 'per_answer_std': np.float64(0.015906469699511084), 'per_prompt_accuracy': np.float64(0.4720812182741117), 'per_prompt_std': np.float64(0.415861821515785)}
0.45454545454545453
{'per_answer_accuracy': np.float64(0.45454545454545453), 'per_answer_std': np.float64(0.023737862175776978), 'per_prompt_accuracy': np.float64(0.45454545454545453), 'per_prompt_std': np.float64(0.43599775674554486)}

In [127]:
for key in correctness_response_length_list.keys():
    # Each entry is (question, correctness, response_length)
    correctness_values = [correct for _, correct, _ in correctness_response_length_list[key]]
    response_lengths = [length for _, _, length in correctness_response_length_list[key]]
    print("Data Source: ", key)
    print("Accuracy: ", np.mean(correctness_values))
    # print("Accuracy Std: ", np.std(correctness_values, ddof=1) if len(correctness_values) > 1 else 0.0)
    print("Response Length: ", np.mean(response_lengths))
    # print("Response Length Std: ", np.std(response_lengths, ddof=1) if len(response_lengths) > 1 else 0.0)
    print("Number of Samples: ", len(correctness_response_length_list[key]))
    # For AUC-OAA, need (correctness, response_length) tuples
    aucoaa_data = [(_, correct, length) for _, correct, length in correctness_response_length_list[key]]
    print("AUC-OAA: ", compute_aucoaa(aucoaa_data))
    print("--------------------------------")


Data Source:  bbeh
Accuracy:  0.18405511811023623
Response Length:  5440.876968503937
Number of Samples:  2032
AUC-OAA:  0.1128701425069317
--------------------------------


In [8]:
for key in correctness_response_length_list.keys():
    print("Data Source: ", key)
    print("Accuracy: ", np.mean([correct for correct, _ in correctness_response_length_list[key]]))
    print("Response Length: ", np.mean([length for _, length in correctness_response_length_list[key]]))
    print("Number of Samples: ", len(correctness_response_length_list[key]))
    print("AUC-OAA: ", compute_aucoaa(correctness_response_length_list[key]))
    print("--------------------------------")

Data Source:  math-amc
Accuracy:  0.7951807228915663
Response Length:  4156.530120481928
Number of Samples:  415
AUC-OAA:  0.5376238678997872
--------------------------------
Data Source:  gpqa
Accuracy:  0.4720812182741117
Response Length:  4193.119796954315
Number of Samples:  985
AUC-OAA:  0.30289823809497224
--------------------------------
Data Source:  math-aime
Accuracy:  0.45454545454545453
Response Length:  6651.779545454546
Number of Samples:  440
AUC-OAA:  0.24861373632365236
--------------------------------


In [26]:
line['response']

'<think>\nOkay, so I need to figure out which of the given options A-J have the correct adjective order based on the example sentences provided. The user mentioned that in this variant of English, the adjectives follow a specific order. From the examples given, I need to determine what that order is.\n\nLooking at the example sentences, like "ancient key" (1), "pyramidal earring" (2), "repulsive normal-size hammer" (3), etc., I need to find a pattern in the adjectives. The correct adjective order in these examples probably follows a certain sequence, such as opinion, size, age, material, purpose, and then other descriptors.\n\nLet me try to deduce the order based on the examples. For instance, in (3) "repulsive normal-size hammer", the adjectives are "repulsive" (opinion), "normal-size" (size), and then "hammer" (noun). Similarly, (4) "red snorkeling bag" has "red" (color), "snorkeling" (purpose), then "bag". Wait, but that seems different. Wait, maybe the order is opinion, size, age, 

In [10]:
len(tokenizer.encode(line['response']))

4407

In [37]:
import json
## read jsonl
with open('your_file.jsonl', 'r') as f:
    data = [json.loads(line) for line in f]


FileNotFoundError: [Errno 2] No such file or directory: 'your_file.jsonl'

In [1]:
from datasets import load_dataset
train_dataset = load_dataset('/nas-ssd2/joykirat/code/verl-fork/baselines/LC-R1/dataset', split='train')

Generating train split:   0%|          | 0/2499 [00:00<?, ? examples/s]

In [2]:
train_dataset

Dataset({
    features: ['problem', 'solution'],
    num_rows: 2499
})

In [3]:
train_dataset[0]

{'problem': 'Let $x$ , $y$ and $z$ all exceed $1$ and let $w$ be a positive number such that $\\log_xw=24$ , $\\log_y w = 40$ and $\\log_{xyz}w=12$ . Find $\\log_zw$ .',
 'solution': '60'}

In [9]:
## load parquet dataset
import pandas as pd
dataset = pd.read_parquet('/nas-ssd2/joykirat/code/verl-fork/verl/scripts/data/dapo-17k/train.parquet')
dataset.head()

,data_source,prompt,ability,reward_model,extra_info,uuid
0,DAPO-Math-17k,"[{'content': 'In triangle $ABC$, $\sin \angle ...",math,"{'ground_truth': '34', 'style': 'rule-lighteva...","{'index': 0, 'split': 'train'}",d2ccd328-e712-4d6b-b909-a7956589db4e
1,DAPO-Math-17k,[{'content': 'Let $ABCD$ be a unit square in t...,math,"{'ground_truth': '113', 'style': 'rule-lightev...","{'index': 1, 'split': 'train'}",5fb8b773-1e5f-48ab-b5ce-8d2ba6ed53cc
2,DAPO-Math-17k,"[{'content': 'Let $a, b, c$ be distinct number...",math,"{'ground_truth': '-3', 'style': 'rule-lighteva...","{'index': 2, 'split': 'train'}",974a3162-fb31-4720-86ab-6a3738ae84e7
3,DAPO-Math-17k,[{'content': 'There are $7$ boxes arranged in ...,math,"{'ground_truth': '3', 'style': 'rule-lighteval...","{'index': 3, 'split': 'train'}",c47003a1-29c7-46ad-b566-977ba9018acb
4,DAPO-Math-17k,[{'content': 'Let $S$ be the set of triples $(...,math,"{'ground_truth': '37', 'style': 'rule-lighteva...","{'index': 4, 'split': 'train'}",f8e041a5-2848-4dfb-99c4-74aa70c8d211


In [10]:
# convert this dataset into a dataset
from datasets import Dataset
dataset = Dataset.from_pandas(dataset)


In [12]:
dataset

Dataset({
    features: ['data_source', 'prompt', 'ability', 'reward_model', 'extra_info', 'uuid'],
    num_rows: 20000
})

In [13]:
dataset = load_dataset('parquet', data_files='/nas-ssd2/joykirat/code/verl-fork/verl/scripts/data/dapo-17k/train.parquet')

Generating train split: 0 examples [00:00, ? examples/s]

In [17]:
### remove some columsn from the dataset
dataset = dataset.remove_columns(['data_source', 'ability', 'extra_info', 'uuid'])

In [21]:
dataset['train'][0]

{'prompt': [{'content': "In triangle $ABC$, $\\sin \\angle A = \\frac{4}{5}$ and $\\angle A < 90^\\circ$. Let $D$ be a point outside triangle $ABC$ such that $\\angle BAD = \\angle DAC$ and $\\angle BDC = 90^\\circ$. Suppose that $AD = 1$ and that $\\frac{BD}{CD} = \\frac{3}{2}$. If $AB + AC$ can be expressed in the form $\\frac{a\\sqrt{b}}{c}$ where $a, b, c$ are pairwise relatively prime integers, find $a + b + c$. Let's think step by step and output the final answer within \\boxed{}.",
   'role': 'user'}],
 'reward_model': {'ground_truth': '34', 'style': 'rule-lighteval/MATH_v2'}}

In [22]:
dataset = dataset['train']

In [24]:
# create a new column with problem name from prompt
dataset = dataset.add_column('problem', [d['prompt'][0]['content'].replace("Let's think step by step and output the final answer within \\boxed{}.", '').strip() for d in dataset])


In [27]:
dataset = dataset.add_column('solution', [d['reward_model']['ground_truth'] for d in dataset])

In [29]:
dataset = dataset.remove_columns(['prompt', 'reward_model'])

In [30]:
dataset

Dataset({
    features: ['problem', 'solution'],
    num_rows: 20000
})

In [34]:
### save the dataset locally in dataset format '/nas-ssd2/joykirat/code/verl-fork/baselines/LC-R1/dapo-17k'
from datasets import load_dataset, load_from_disk
# dataset.save_to_disk('/nas-ssd2/joykirat/code/verl-fork/baselines/LC-R1/dapo-17k')

In [35]:
# load the dataset from the local dataset format
dataset = load_from_disk('/nas-ssd2/joykirat/code/verl-fork/baselines/LC-R1/dapo-17k')

In [6]:
# dataset_name = 'full'
# folder_name = f'{dataset_name}_test_dataset'
from datasets import load_dataset
import pandas as pd
dataset_name = 'overthink'

df = pd.read_parquet(f"/nas-ssd2/joykirat/code/verl-fork/verl/scripts/data/{dataset_name}_test_dataset/test.parquet")

print(len(df))

607
